<a href="https://colab.research.google.com/github/SyedHasnat/CNNs-LSTMs-Time-Series/blob/main/2_How_to_Develop_CNNs_for_Time_Series_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import array,hstack
import pandas as pd
import matplotlib as pt
import numpy as np

# Multivariate CNN Models Data pretaration

## Multiple Input Series

In [ ]:
in_seq1=array([10,20,30,40,50,60,70,80,90])
in_seq2=array([15,25,35,45,55,65,75,85,95])
out_seq=array([in_seq1[i]+in_seq1[i] for i in range(len(in_seq1))])


In [ ]:
in_seq1

array([[10],
       [20],
       [30],
       [40],
       [50],
       [60],
       [70],
       [80],
       [90]])

In [ ]:
in_seq1=in_seq1.reshape(len(in_seq1),1) #changing row to coumn
in_seq2=in_seq2.reshape(len(in_seq2),1)
out_seq=out_seq.reshape(len(out_seq),1)

In [ ]:
#lets make a dataset 
dataset = hstack((in_seq1,in_seq2,out_seq))

In [ ]:
#for i in range(len(dataset)):
#    print(dataset[i])
#or
print(dataset)

[[ 10  15  20]
 [ 20  25  40]
 [ 30  35  60]
 [ 40  45  80]
 [ 50  55 100]
 [ 60  65 120]
 [ 70  75 140]
 [ 80  85 160]
 [ 90  95 180]]


In [ ]:
dataset.shape

(9, 3)

# For CNN we need 3D lets reshape it

In [ ]:
dataset=dataset.reshape(dataset.shape[0],dataset.shape[1],1)

In [ ]:
dataset.shape

(9, 3, 1)

# Defining a fuction which will organize a data set for CNN

In [ ]:
#split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)



in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(7, 3, 2) (7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


# Trying to explain the above code

In [ ]:
b=X[1]

In [ ]:
b.shape# chapter 6 ki mong sara 3x1 wo oss 3x2 dai
# da yo column ba mo 3 values warkra nu pa aghi na bad ba output wo oss 2 coulumns dy da aghi 3 , 3 warkao....

(3, 2)

In [ ]:

X, y = list(), list()
for i in range(len(dataset)):
    # find the end of this pattern
    end_ix = i +3
    # check if we are beyond the dataset
    if end_ix > len(dataset):
        break
# gather input and output parts of the pattern
    #dataset[i:end_ix, :-1] matlab dataset[i na wakhli tar end_ix pori , :-1 except last column]
    #dataset[end_ix-1, -1] matlab dataset[end_ix-1 matlb 3-1=2, -1 matlab total 3 columns dy -1=2.....matlab (2,2)]
    
    #seq_x, seq_y = dataset[i:end_ix, :-1], dataset[end_ix-1, -1] #-1 means don't store last column if -2 so last 2 columns
    seq_x, seq_y = dataset[i:end_ix, :2], dataset[end_ix-1, 2]   #both are samw
    X.append(seq_x)
    y.append(seq_y)
for i in range(len(X)):
    print(X[i],y[i])

[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [ ]:
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [ ]:
seq1=([10,20,30,40,50,60,70,80,90])
seq2=([15,25,35,45,55,65,75,85,95])
seq3=([seq1[i]+seq2[i] for i in range(len(seq1))])

In [ ]:
seq1=array(seq1)
seq2=array(seq2)
seq3=array(seq3)

In [ ]:
seq1=seq1.reshape((len(seq1),1))
seq2=seq2.reshape((len(seq2),1))
seq3=seq3.reshape((len(seq3),1))

In [ ]:
dataset=hstack((seq1,seq2,seq3))

In [ ]:
x,y=list(),list()
for i in range(len(dataset)):
    endx_i=i+3
    if endx_i>len(dataset):
        break
    seq_x=dataset[i:endx_i,:2]
    seq_y=dataset[endx_i-1,2]
    x.append(seq_x)
    y.append(seq_y)
for i in range(len(x)):
    print(x[i],y[i])

[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [ ]:
x=array(x)
print(x.shape)
print(x.size) #samples=7 time steps= 3 featutres= 2

(7, 3, 2)
42


# Multivariate CNN Model

In [ ]:
# multivariate cnn example

###################################################importing modules###############################################

from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

###################################################function to split dataset#######################################

# split a multivariate sequence into samples
def split_sequences(dataset, n_steps):
    X, y = list(), list()
    for i in range(len(dataset)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(dataset):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = dataset[i:end_ix, :-1], dataset[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

###################################################define input sequence###########################################

in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

###################################################convert to [rows, columns] structure############################

in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

###################################################horizontally stack columns#####################################

dataset = hstack((in_seq1, in_seq2, out_seq))

###################################################choose a number of time steps##################################

n_steps = 3

###################################################convert into input/output######################################

X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e. g. 2
n_features = X.shape[2]

###################################################define model##################################################

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu' , input_shape=(n_steps,
n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu' ))
model.add(Dense(1))
model.compile(optimizer='adam' , loss='mse' )

###################################################fit model#####################################################

model.fit(X, y, epochs=1000, verbose=0)

###################################################demonstrate prediction########################################

x_input = array([[80, 85], [90, 95], [100, 105]]) #input is 3x2 while in the previous chpater it ws 3x1
print(x_input.shape)
print(x_input.size)
x_input = x_input.reshape((1, n_steps, n_features))
print(x_input.shape)
print(x_input.size)
yhat = model.predict(x_input, verbose=0)
print(yhat)

(3, 2)
6
(1, 3, 2)
6
[[207.41014]]


# Multi-headed CNN Model
There is another, more elaborate way to model the problem. Each input series can be handled by
a separate CNN and the output of each of these submodels can be combined before a prediction
is made for the output sequence. We can refer to this as a multi-headed CNN model. 

In [ ]:
# multivariate cnn example

###################################################importing modules###############################################

from numpy import array
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate

###################################################function to split dataset#######################################

# split a multivariate sequence into samples
def split_sequences(dataset, n_steps):
    X, y = list(), list()
    for i in range(len(dataset)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(dataset):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = dataset[i:end_ix, :-1], dataset[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

###################################################define input sequence###########################################

in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

###################################################convert to [rows, columns] structure############################

in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

###################################################horizontally stack columns#####################################

dataset = hstack((in_seq1, in_seq2, out_seq))

###################################################choose a number of time steps##################################

# one time series per head
n_features = 1

###################################################separate input data############################################

X1 = X[:, :, 0].reshape(X.shape[0], X.shape[1], n_features)
X2 = X[:, :, 1].reshape(X.shape[0], X.shape[1], n_features)

###################################################first input model##############################################

visible1 = Input(shape=(n_steps, n_features))
cnn1 = Conv1D(filters=64, kernel_size=2, activation='relu' )(visible1)
cnn1 = MaxPooling1D(pool_size=2)(cnn1)
cnn1 = Flatten()(cnn1)

###################################################second input model############################################

visible2 = Input(shape=(n_steps, n_features))
cnn2 = Conv1D(filters=64, kernel_size=2, activation='relu' )(visible2)
cnn2 = MaxPooling1D(pool_size=2)(cnn2)
cnn2 = Flatten()(cnn2)

###################################################merge input models############################################

merge = concatenate([cnn1, cnn2])
dense = Dense(50, activation='relu' )(merge)
output = Dense(1)(dense)
model = Model(inputs=[visible1, visible2], outputs=output)
model. compile(optimizer='adam' , loss='mse' )

###################################################fit model#####################################################

model.fit([X1, X2], y, epochs=1000, verbose=0)

###################################################fdemonstrate prediction#######################################

x_input = array([[80, 85], [90, 95], [100, 105]])
x1 = x_input[:, 0].reshape((1, n_steps, n_features))
x2 = x_input[:, 1].reshape((1, n_steps, n_features))
yhat = model.predict([x1, x2], verbose=0)
print(yhat)

[[208.24243]]


# We may want to predict the value for each of the three time series for the next time step

# First we will prepare data to train the model

In [ ]:
# multivariate cnn example

###################################################importing modules###############################################

from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

###################################################function to split dataset#######################################

# split a multivariate sequence into samples
def split_sequences(dataset, n_steps):
    X, y = list(), list()
    for i in range(len(dataset)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(dataset)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = dataset[i:end_ix, :], dataset[end_ix, :] # change #dataset[i:end_ix, :-1], dataset[end_ix-1, -1]
        #dataset[i:end_ix, :] all columns 
        #dataset[end_ix, :] only 3rd row and all columns
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

###################################################define input sequence###########################################

in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

###################################################convert to [rows, columns] structure############################

in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

###################################################horizontally stack columns#####################################

dataset = hstack((in_seq1, in_seq2, out_seq))

###################################################choose a number of time steps##################################

n_steps = 3

###################################################convert into input/output######################################

X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e. g. 2
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])
#n_features = X.shape[2]


(6, 3, 3) (6, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]


In [ ]:
# multivariate cnn example

###################################################importing modules###############################################

from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

###################################################function to split dataset#######################################

# split a multivariate sequence into samples
def split_sequences(dataset, n_steps):
    X, y = list(), list()
    for i in range(len(dataset)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(dataset)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = dataset[i:end_ix, :], dataset[end_ix, :] # change #dataset[i:end_ix, :-1], dataset[end_ix-1, -1]
        #dataset[i:end_ix, :] all columns 
        #dataset[end_ix, :] only 3rd row and all columns
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

###################################################define input sequence###########################################

in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

###################################################convert to [rows, columns] structure############################

in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

###################################################horizontally stack columns#####################################

dataset = hstack((in_seq1, in_seq2, out_seq))

###################################################choose a number of time steps##################################

n_steps = 3

###################################################convert into input/output######################################

X, y = split_sequences(dataset, n_steps)

n_features = X.shape[2]

###################################################define model##################################################

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu' , input_shape=(n_steps,
n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu' ))
model.add(Dense(n_features))
model. compile(optimizer='adam' , loss='mse' )

###################################################fit model#####################################################

model.fit(X, y, epochs=3000, verbose=0)

###################################################demonstrate prediction#######################################

x_input = array([[70,75,145], [80,85,165], [90,95,185]])
print(x_input.shape)
print(x_input.size)
x_input = x_input.reshape((1, n_steps, n_features))
print(x_input.shape)
print(x_input.size)
yhat = model.predict(x_input, verbose=0)
print(yhat)

(3, 3)
9
(1, 3, 3)
9
[[101.38163 106.30747 207.05624]]


# To be continue.....2.1